In [1]:
import h5py
from sklearn.cross_validation import train_test_split
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Convolution2D, MaxPooling2D, Flatten
from keras.optimizers import SGD

Using TensorFlow backend.


In [2]:
# Load data
with h5py.File("data/split_data.hdf5", "r") as f:
    X_train = f["X_train"][:]
    X_test = f["X_test"][:]
    artist_train = f["artist_train"][:]
    artist_test = f["artist_test"][:]
    song_train = f["song_train"][:]
    song_test = f["song_test"][:]
    y_train = f["Y_train"][:]
    y_test = f["Y_test"][:]

X_train.shape = (264189, 1, 200, 12)   # Reshaped to fit model
X_test.shape = (113225, 1, 200, 12)

In [24]:
# Convnet
model = Sequential()
model.add(Convolution2D(20, 10, 1, init='uniform', border_mode='valid', input_shape=(1, 200, 12)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,1),border_mode='valid'))
model.add(Convolution2D(20, 10, 1, init='uniform', border_mode='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,1),border_mode='valid'))

model.add(Flatten())
model.add(Dense(200, init='uniform'))
model.add(Activation('relu'))
# model.add(Dropout(0.5))
model.add(Dense(200, init='uniform'))
model.add(Activation('relu'))
# model.add(Dropout(0.5))
model.add(Dense(10, init='uniform'))
model.add(Activation('tanh'))

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss="mean_squared_error",
              optimizer=sgd)

hist = model.fit(X_train, y_train, nb_epoch=1, batch_size=100, validation_data=(X_test,y_test))

Train on 264189 samples, validate on 113225 samples
Epoch 1/1
 15800/264189 [>.............................] - ETA: 522s - loss: 0.0013

KeyboardInterrupt: 

In [34]:
# Save model
model.save("model.h5")

In [35]:
# Load model
model = load_model('model.h5')

In [36]:
model.predict(X_test[1000:1050])

array([[ -1.69096084e-03,   1.21529454e-04,  -6.43433013e-04,
          4.84977121e-04,   9.36862605e-04,   2.77492916e-03,
          3.88558547e-04,   4.41789191e-04,   3.84403771e-04,
          2.53665174e-04],
       [ -1.70619308e-03,   1.08458582e-04,  -6.43749663e-04,
          4.85542085e-04,   9.53167910e-04,   2.74469494e-03,
          3.88114655e-04,   4.52448119e-04,   3.81208054e-04,
          2.17240900e-04],
       [ -1.82887586e-03,   8.30435456e-05,  -7.54271634e-04,
          5.08495606e-04,   9.79636097e-04,   2.71443813e-03,
          4.55175614e-04,   4.75224340e-04,   3.92156333e-04,
          3.21255589e-04],
       [ -1.77632971e-03,  -9.08105576e-05,  -1.18583732e-03,
          3.49914713e-04,   6.00006373e-04,   2.78272806e-03,
          2.51814432e-04,   6.95114606e-04,   1.99453643e-04,
          5.94103651e-04],
       [ -1.70619308e-03,   1.08458582e-04,  -6.43749663e-04,
          4.85542085e-04,   9.53167910e-04,   2.74469494e-03,
          3.88114655e-04